# Heterogeneous Agents - Solving the Aiyagari problem - OSE - Tony Smith 

## Jeanne Sorin

### Calibrate the transition probability

In [1]:
import numpy as np
from scipy.optimize import root
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy import interpolate


In [61]:
# Define the employment status
ϵ = np.array((0.0, 0.0))

# Calibrate the transition probability
A = np.array([0.9, 0.1])
Trans= np.array([[0.9,0.1], [0.9, 0.1]] )
Ubar = 0.1
α = 0.36
β = 0.96
δ = 0.06


def rf(k, u):
    r = α*k**(α-1)*(1-u)**(1-α)
    return r
    
def wf(k, u):
    w = (1-α)*k**(α) * (1-u)**(-α)
    return w

# Define the FOC of the HH

def utility(c):
    if c > 0:
        U = np.log(c)
    if c <= 0:
        U = -99999999
    return U

def uprime(c):
    Up = -1/(c)
    return Up

'''
From the intertemporal EE, and the the definition of k and w as a function of k
'''

def find_K_SS(α, β, δ, u):
    k_ss = ((1/β - 1 + δ) / (α*(1 - u) **(1-α))) ** (1 / (α-1)) 
    return(k_ss)

Kss = find_K_SS(α, β, δ, Ubar)
Kss

    

6.4900587533395635

### Compute the steady state equilibrium value of aggregate capital kss*

#### 1. Guess on kss from the representative-agent economy in ss

In [62]:
def interpolant(x,Kgrid,Vp):
    t = interpolate.interp1d(Kgrid,Vp,fill_value="extrapolate")
    return t(x)


def Bellman(kp, *params):
    r, w, Kgrid, Vp, ϵ, ik, ie, Trans = params
    #print("Before U")
    today = utility(r*Kgrid[ik] + w*ϵ[ie] + (1-δ)*Kgrid[ik] - kp)
    #print("Computed utility ", today)
    
    fut = 0
    #print("Computing future")
    for i in range(Nϵ):
        #print("fut is ", fut)
        fut +=  Trans[ie,i]*interpolant(Kp[ik, ie],Kgrid,Vp[:,i])  
        
    #print("Computed future ", fut)

    Value = today + β * fut
    #print("Computed total")
    return - Value



#### 2. Restrict k to lie on a grid of N = 200 points in the interval [0.001, 40] and then iterate on the Bellman equation to find v(k, e, kss) at each of the pairs in k*e

Use value function iteration for the policy function to know how k' depends on k.

In [64]:
n = 50
Nϵ = 2
tol = 1e-6
ϵ = np.array([1,0])
maxiter = 2000

# Make Grid

Kgrid = np.linspace(1., 10., n)

Klow = 1
Khigh = 9

Vp = Klow*np.ones([n, Nϵ])
Kp= Klow*np.ones([n, Nϵ])
Vp_new = np.zeros([n, Nϵ])
Kp_new = np.zeros([n, Nϵ])



#### Run function

iter = 0
epsi = 1

r = rf(Kss, Ubar)
w = wf(Kss, Ubar) 

while (epsi > tol) & (iter < maxiter):
    
    
    for ik in range(n):
        
        for ie in range(Nϵ):
            
            params1 = r, w, Kgrid, Vp, ϵ, ik, ie, Trans
            #print("kp is ", Kp[ik, ie])
            Solve = minimize(Bellman, Kp[ik, ie], args=(params1))
            Kp_new[ik, ie] = Solve.x
            #print("KP new is ",Kp_new[ik, ie] )
            Vp_new[ik, ie] = - Solve.fun

    iter += 1

    epsi = ((Vp_new - Vp) ** 2).sum()
    print("Error term: ", epsi)
    if epsi < tol:
        break
    #Update
    Kp = Kp_new*1.
    Vp = Vp_new*1.
    iter=iter+1

        

Error term:  13925.917141780097
Error term:  258396841437.774
Error term:  6.087458333892789e+20
Error term:  1.433538754126116e+30
Error term:  3.3758479537410315e+39
Error term:  7.94980210613346e+48
Error term:  1.8721030802541847e+58
Error term:  4.408625392565728e+67
Error term:  1.0381895130120876e+77
Error term:  2.4448379459634543e+86
Error term:  5.757361740903293e+95
Error term:  1.3558041452336208e+105
Error term:  3.192790314308561e+114
Error term:  7.518718707993057e+123
Error term:  1.7705870240391024e+133
Error term:  4.169564697722897e+142
Error term:  9.818929842170267e+151
Error term:  2.3122649541357244e+161
Error term:  5.445164905e+170
Error term:  1.2822847481043058e+180
Error term:  3.0196590992333653e+189
Error term:  7.111011098793103e+198
Error term:  1.674575744659281e+208
Error term:  3.9434672308092835e+217
Error term:  9.286491727867902e+226
Error term:  2.1868808224903273e+236
Error term:  5.149897153759692e+245
Error term:  1.212751990028466e+255
Error t

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in square


Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf
Error term:  inf


KeyboardInterrupt: 

In [ ]:
print(Vp)


In [ ]:
print (params)
print (params1)

In [ ]:
Vguess = Sol[0]
Kpguess = Sol[1]

print("Vguess is ", Vguess)
print("Kpguess is ", Kpguess)

In [ ]:
plt.plot(krep, Kpguess[0], label="Employed at t")
plt.plot(krep, Kpguess[1], label="Unemployed at t")
plt.legend()
plt.title("Policy Function")
plt.show()


plt.plot(krep, Vguess[0,:], label="Employed at t")
plt.plot(krep, Vguess[1,:], label="Unemployed at t")
plt.legend()
plt.title("Value Function")
plt.show()

#### 3. Simulate a long time series for a typical consumer using the optimal decision rule (computed in 2.) and the law of motion for e

In [ ]:
Kguess

#### 4. Calculate average capital holdings kss^ using the simulated data